In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.append("../src/SpaceMAP/")
from _spacemap import SpaceMAP

sys.path.append("../run/")
from experimet_config import Config

cfg = Config()

In [2]:
use_columns = ['id', 'file_name', 'folder_name']
merged_df = pd.read_csv('../data/table/merged.csv')[use_columns].dropna().reset_index(drop=True)

geometry_df = pd.read_excel('../data/table/geometry.xlsx', index_col=0).T.reset_index().rename(columns={'index': 'file_name'})

merged_df = pd.merge(merged_df, geometry_df, on='file_name', how='left').dropna().reset_index(drop=True)

In [3]:
geometry_array = merged_df[cfg.geometry_attribute].values
img_paths = [f'../../../data/img/{i}.jpg' for i in merged_df['folder_name'].values]

In [7]:
# let's see the decomposed latent space

# 1. PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
reduced_latent_codes = pca.fit_transform(geometry_array)
np.save('../outputs/decompose-analysis/geometry/reduced_geometry-PCA.npy', reduced_latent_codes)

# 2. tSNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=30.0, n_iter=1000)
reduced_latent_codes = tsne.fit_transform(geometry_array)
np.save('../outputs/decompose-analysis/geometry/reduced_geometry-tSNE.npy', reduced_latent_codes)

# 3. UMAP
from umap import UMAP
umap = UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean')
reduced_latent_codes = umap.fit_transform(geometry_array)
np.save('../outputs/decompose-analysis/geometry/reduced_geometry-UMAP.npy', reduced_latent_codes)

"""
# 4. SpaceMAP
spacemap = SpaceMAP(verbose=False)
reduced_latent_codes = spacemap.fit_transform(geometry_array)
np.save('../outputs/decompose-analysis/geometry/reduced_geometry-spaceMAP.npy', reduced_latent_codes)
"""

"\n# 4. SpaceMAP\nspacemap = SpaceMAP(verbose=False)\nreduced_latent_codes = spacemap.fit_transform(geometry_array)\nnp.save('../outputs/decompose-analysis/geometry/reduced_geometry-spaceMAP.npy', reduced_latent_codes)\n"

In [8]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper

methods = [
    # 'spaceMAP',
    'UMAP',
    'PCA',
    'tSNE',
]

for method in methods:
    reduced_latent_codes = np.load(f'../outputs/decompose-analysis/geometry/reduced_geometry-{method}.npy')

    digits_df = pd.DataFrame(reduced_latent_codes, columns=['x', 'y'])
    digits_df['image'] = img_paths
    digits_df['digit'] = digits_df['image'].apply(lambda x: x.split('/')[-1].split('.')[0])

    datasource = ColumnDataSource(digits_df)

    #color_mapping = CategoricalColorMapper(factors=[str(9 - x) for x in digits.target_names],
    #                                       palette=Spectral10)

    plot_figure = figure(
        title='{} projection of the Digits dataset'.format(method),
        outer_width=600,
        outer_height=600,
        tools=('pan, wheel_zoom, reset')
    )

    plot_figure.add_tools(HoverTool(tooltips="""
    <div>
        <div>
            <img src='@image' style='float: left; margin: 5px 5px 5px 5px; width:300px;'/>
        </div>
        <div>
            <span style='font-size: 16px; color: #224499'>Digit:</span>
            <span style='font-size: 18px'>@digit</span>
        </div>
    </div>
    """))

    plot_figure.circle(
        'x',
        'y',
        source=datasource,
        # color=dict(field='digit', transform=color_mapping),
        line_alpha=0.6,
        fill_alpha=0.6,
        size=4
    )
    show(plot_figure)

    from bokeh.io import save

    save(plot_figure, '../outputs/decompose-analysis/geometry/geometry-mapping-{}.html'.format(method))

/tmp/ipykernel_29346/4067847063.py:55: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(plot_figure, '../outputs/decompose-analysis/geometry/geometry-mapping-{}.html'.format(method))
/tmp/ipykernel_29346/4067847063.py:55: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(plot_figure, '../outputs/decompose-analysis/geometry/geometry-mapping-{}.html'.format(method))
/tmp/ipykernel_29346/4067847063.py:55: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(plot_figure, '../outputs/decompose-analysis/geometry/geometry-mapping-{}.html'.format(method))
/tmp/ipykernel_29346/4067847063.py:55: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(plot_figure, '../outputs/decomp